**Main imports**

In [7]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from IPython import display as disp

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

**Import dataset**

In [ ]:
# helper function to make getting another batch of data easier
def cycle(iterable):
    while True:
        for x in iterable:
            yield x

class_names = ['apple','aquarium_fish','baby','bear','beaver','bed','bee','beetle','bicycle','bottle','bowl','boy','bridge','bus','butterfly','camel','can','castle','caterpillar','cattle','chair','chimpanzee','clock','cloud','cockroach','couch','crab','crocodile','cup','dinosaur','dolphin','elephant','flatfish','forest','fox','girl','hamster','house','kangaroo','computer_keyboard','lamp','lawn_mower','leopard','lion','lizard','lobster','man','maple_tree','motorcycle','mountain','mouse','mushroom','oak_tree','orange','orchid','otter','palm_tree','pear','pickup_truck','pine_tree','plain','plate','poppy','porcupine','possum','rabbit','raccoon','ray','road','rocket','rose','sea','seal','shark','shrew','skunk','skyscraper','snail','snake','spider','squirrel','streetcar','sunflower','sweet_pepper','table','tank','telephone','television','tiger','tractor','train','trout','tulip','turtle','wardrobe','whale','willow_tree','wolf','woman','worm',]

train_transform = transforms.Compose([
    # resize all images to 32x32
    transforms.Resize((32, 32)),
    # random flipping
#     transforms.RandomHorizontalFlip(),
#     # randomly rotate images up to 5 degrees
#     transforms.RandomRotation(5),
#     # affine transformations with shear
#     transforms.RandomAffine(degrees=10, shear=5),
#     # Gaussian blur
#     transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 1.0)),
    # convert to tensor
    transforms.ToTensor()
])

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR100('data', train=True, download=True, transform=train_transform),
    batch_size=64, drop_last=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR100('data', train=False, download=True, transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])),
    batch_size=64, drop_last=True)

train_iterator = iter(cycle(train_loader))
test_iterator = iter(cycle(test_loader))

print(f'> Size of training dataset {len(train_loader.dataset)}')
print(f'> Size of test dataset {len(test_loader.dataset)}')

**View some of the test dataset**

In [ ]:
# let's view some of the training data
plt.rcParams['figure.dpi'] = 100
x,t = next(train_iterator)
x,t = x.to(device), t.to(device)
plt.imshow(torchvision.utils.make_grid(x).cpu().numpy().transpose(1, 2, 0), cmap=plt.cm.binary)
plt.show()

# Model

## DCGAN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim  
        self.img_channels = 3  # RGB images
        self.model = nn.Sequential(
            nn.ConvTranspose2d(in_channels=self.latent_dim, out_channels=96, kernel_size=4, stride=1, padding=0, bias=False),  # Output: 96x4x4
            nn.BatchNorm2d(num_features=96),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=96, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 64x8x8
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=64, out_channels=48, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 48x16x16
            nn.BatchNorm2d(num_features=48),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=48, out_channels=self.img_channels, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 3x32x32
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

    def sample(self, z):
        with torch.no_grad():
            samples = self.forward(z)
        return samples


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.img_channels = 3  # RGB images
        self.model = nn.Sequential(
            nn.Conv2d(self.img_channels, 64, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 64x16x16
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 128x8x8
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 256x4x4
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 1, kernel_size=4, stride=1, padding=0, bias=False),  # Output: 1x1x1
            nn.Sigmoid()
        )

    def forward(self, img):
        output = self.model(img)
        return output.view(-1)



# {'g_lr': 0.0006828838288502215, 'd_lr': 0.000275880375280674, 'step_size': 7, 'gamma': 0.1152882158531075} FID 44.55

# Hyperparameters
latent_dim = 100
img_channels = 3
step_size = 7 
gamma = 0.1152882158531075     # Reduce LR by multiplying by gamma

# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)
# Optimisers and loss function
optimiser_G = optim.Adam(generator.parameters(), lr=0.0006828838288502215, betas=(0.5, 0.999))
optimiser_D = optim.Adam(discriminator.parameters(), lr=0.000275880375280674, betas=(0.5, 0.999))
criterion = nn.BCELoss()

# Schedulers
scheduler_gen = StepLR(optimiser_G, step_size=step_size, gamma=gamma)
scheduler_disc = StepLR(optimiser_D, step_size=step_size, gamma=gamma)


print(f'> Number of generator parameters {len(torch.nn.utils.parameters_to_vector(generator.parameters()))}')
print(f'> Number of discriminator parameters {len(torch.nn.utils.parameters_to_vector(discriminator.parameters()))}')
total_params = len(torch.nn.utils.parameters_to_vector(generator.parameters())) + len(torch.nn.utils.parameters_to_vector(discriminator.parameters()))
print(total_params)
if total_params > 1000000:
    print("> Warning: you have gone over your parameter budget and will have a grade penalty!")

In [103]:
steps = 0
batch_size = 64
epochs = 0

params = {
    'batch_size': 64,
    'latent_dim': latent_dim,
    'n_channels': img_channels
}

In [ ]:
while steps < 50000:
    loss_arr_D = np.zeros(0)
    loss_arr_G = np.zeros(0)

    for _ in range(1000):
        x, _ = next(train_iterator)

        real_imgs = x.cuda()

        # Train Discriminator
        z = torch.randn(batch_size, latent_dim, 1, 1).to(device)
        fake_imgs = generator(z)

        real_labels = torch.ones(batch_size).to(device)
        fake_labels = torch.zeros(batch_size).to(device)

        # Real images loss
        real_loss = criterion(discriminator(real_imgs), real_labels)
        # Fake images loss
        # Make sure the output shape is [batch_size, 1] (scalar per image)
        fake_loss = criterion(discriminator(fake_imgs.detach()), fake_labels)
        d_loss = real_loss + fake_loss

        optimiser_D.zero_grad()
        d_loss.backward()
        optimiser_D.step()

        # Train Generator
        g_loss = criterion(discriminator(fake_imgs), real_labels)

        optimiser_G.zero_grad()
        g_loss.backward()
        optimiser_G.step()

        steps += 1

        loss_arr_D = np.append(loss_arr_D, d_loss.item())
        loss_arr_G = np.append(loss_arr_G, g_loss.item())

        if steps >= 50000:
            break

    print(f"Steps: {steps}, Loss D: {loss_arr_D.mean():.3f}, Loss G: {loss_arr_G.mean():.3f}")

    # Sample model and visualize results
    generator.eval()
    z = torch.randn(64, latent_dim, 1, 1).cuda()
    samples = generator.sample(z).cpu().detach()
    plt.imshow(torchvision.utils.make_grid(samples).cpu().numpy().transpose(1, 2, 0), cmap=plt.cm.binary)
    plt.show()
    disp.clear_output(wait=True)
    epochs += 1
    generator.train()
    if epochs % step_size == 0:
        scheduler_gen.step()
        scheduler_disc.step()

**Latent interpolations**

In [ ]:
import torch
import matplotlib.pyplot as plt
import torchvision

# Assuming 'batch_size' and 'latent_dim' are defined

# Generate two random latent vectors
z0 = torch.randn(batch_size, latent_dim, 1, 1, device=device)
z7 = torch.randn(batch_size, latent_dim, 1, 1, device=device)

# Prepare the grid for storing interpolated images
grid = []
for i in range(8):
    a = i / 7  
    b = 1 - a
    z_terp = (b * z0) + (a * z7) 

    row = generator.sample(z_terp).detach().cpu()[:8]  
    
    grid.append(row)

# Concatenate the rows to form the grid
grid = torch.cat(grid, dim=0)

# Display the 8x8 grid of interpolated images
plt.rcParams['figure.dpi'] = 100
plt.grid(False)
plt.imshow(torchvision.utils.make_grid(grid, normalize=True).cpu().numpy().transpose(1, 2, 0), cmap=plt.cm.binary)
plt.axis('off')  # Turn off axis
plt.show()


z0 = torch.randn(batch_size, latent_dim, 1, 1,device=device)
z7 = torch.randn(batch_size, latent_dim, 1, 1,device=device)

grid = []
for i in range(8):
    a = i / 7
    b = 1 - a
    z_terp = (b*z0) + (a*z7)
#     print(z_terp)
    row = generator.sample(z_terp).detach().cpu()[:8]
    
#     row = (row+1)/2
    grid.append(row)
grid = torch.concat(grid, dim=0)
plt.rcParams['figure.dpi'] = 100
plt.grid(False)
plt.imshow(torchvision.utils.make_grid(grid).cpu().numpy().transpose(1,2,0), cmap=plt.cm.binary)
# plt.axis(False)
plt.show()

**FID scores**

Evaluate the FID from 10k of your model samples (do not sample more than this) and compare it against the 10k test images. Calculating FID is somewhat involved, so we use a library for it. It can take a few minutes to evaluate. Lower FID scores are better.

In [ ]:
%%capture
!pip install clean-fid

In [106]:
import os
from cleanfid import fid
from torchvision.utils import save_image

In [107]:
# define directories
real_images_dir = 'real_images'
generated_images_dir = 'generated_images'
num_samples = 10000 # do not change

# create/clean the directories
def setup_directory(directory):
    if os.path.exists(directory):
        !rm -r {directory} # remove any existing (old) data
    os.makedirs(directory)

setup_directory(real_images_dir)
setup_directory(generated_images_dir)

# generate and save 10k model samples
num_generated = 0
while num_generated < num_samples:

    # sample from your model, you can modify this
    z = torch.randn(64, latent_dim, 1, 1).cuda()
    samples_batch = generator.sample(z).cpu().detach()

    for image in samples_batch:
        if num_generated >= num_samples:
            break
        save_image(image, os.path.join(generated_images_dir, f"gen_img_{num_generated}.png"))
        num_generated += 1

# save 10k images from the CIFAR-100 test dataset
num_saved_real = 0
while num_saved_real < num_samples:
    real_samples_batch, _ = next(test_iterator)
    for image in real_samples_batch:
        if num_saved_real >= num_samples:
            break
        save_image(image, os.path.join(real_images_dir, f"real_img_{num_saved_real}.png"))
        num_saved_real += 1

In [ ]:
# compute FID
score = fid.compute_fid(real_images_dir, generated_images_dir, mode="clean")
print(f"FID score: {score}")

In [ ]:
!pip install lpips

In [ ]:
import lpips
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


lpips_model = lpips.LPIPS(net='vgg')  
lpips_model.eval() 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lpips_model.to(device)

# New train dataset transformations so that it aligns with how lpips model was trained
lpips_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=lpips_transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

# Sample a batch of 64 images
sample_batch, _ = next(iter(train_loader))  # Get a batch from the train_loader
sample_batch = sample_batch.to(device)  # Move the batch to the appropriate device


lpips_scores = []

with torch.no_grad():
    for idx, sample_img in enumerate(sample_batch):
        sample_img = sample_img.unsqueeze(0)  # Add batch dimension for LPIPS model

       
        for img, label in train_loader:
            img = img.to(device)
            dist = lpips_model(sample_img, img)  # Compute LPIPS score between images

            
            for d in dist.squeeze(0):  
                lpips_scores.append(d.item()) 

# Convert list of LPIPS scores to a tensor
lpips_scores = torch.tensor(lpips_scores)
# Output mean and standard deviation of lpips scores to identify how similar images are to nearest neighbours
print(np.array(lpips_scores).mean(), np.array(lpips_scores).std())


# Hyperparameter Tuning

In [ ]:
# DCGAN
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import optuna 
import os
from cleanfid import fid
from torchvision.utils import save_image
img_channels = 3

class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim  
        self.img_channels = 3  # RGB images
        self.model = nn.Sequential(
            nn.ConvTranspose2d(in_channels=self.latent_dim, out_channels=96, kernel_size=4, stride=1, padding=0, bias=False),  # Output: 96x4x4
            nn.BatchNorm2d(num_features=96),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=96, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 64x8x8
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=64, out_channels=48, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 48x16x16
            nn.BatchNorm2d(num_features=48),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=48, out_channels=self.img_channels, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 3x32x32
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

    def sample(self, z):
        with torch.no_grad():
            samples = self.forward(z)
        return samples


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.img_channels = 3  # RGB images
        self.model = nn.Sequential(
            nn.Conv2d(self.img_channels, 64, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 64x16x16
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 128x8x8
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),  # Output: 256x4x4
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 1, kernel_size=4, stride=1, padding=0, bias=False),  # Output: 1x1x1
            nn.Sigmoid()
        )

    def forward(self, img):
        output = self.model(img)
        return output.view(-1)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def objective(trial):
    # Hyperparameter search space
    g_lr = trial.suggest_loguniform("g_lr", 2.5e-4, 9.5e-4)
    d_lr = trial.suggest_loguniform("d_lr", 2.5e-4, 9.5e-4)
#     latent_dim = trial.suggest_int("latent_dim", 64, 128, step=32)
    step_size = trial.suggest_int("step_size", 5, 20)
    gamma = trial.suggest_float("gamma", 0.1, 0.85)
#     dropout = trial.suggest_float("dropout",0.12,0.25)
#     b1 = trial.suggest_float("b1", 0.475, 0.525)
#     b2 = trial.suggest_float("b2", 0.96, 0.9999)
    b1 = 0.5
    b2 = 0.99
#     print(f"g_lr: {g_lr}, d_lr: {d_lr}, latent_dim: {latent_dim}")
#     print(f"step_size: {step_size_g, step_size_d}, gamma_g: {gamma_g}, gamma_d: {gamma_d})")
    

    latent_dim = 100
#     - {'g_lr': 0.0006828838288502215, 'd_lr': 0.000275880375280674, 'step_size': 7, 'gamma': 0.1152882158531075} FID 44.55
    
    generator = Generator(latent_dim).to(device)
    discriminator = Discriminator().to(device)
    # Optimisers and scheduler
    optimiser_G = optim.Adam(generator.parameters(), lr=g_lr, betas=(b1, b2))
    optimiser_D = optim.Adam(discriminator.parameters(), lr=d_lr, betas=(b1, b2))
    scheduler_gen = torch.optim.lr_scheduler.StepLR(optimiser_G, step_size=step_size, gamma=gamma)
    scheduler_disc = torch.optim.lr_scheduler.StepLR(optimiser_D, step_size=step_size, gamma=gamma)

    criterion = nn.BCELoss()
    
    steps = 0
    epochs = 0
    batch_size = 64
    params = {
        'batch_size': 64,
        'latent_dim': latent_dim,
        'n_channels': img_channels
    }
    
    while steps < 50000:
        loss_arr_D = np.zeros(0)
        loss_arr_G = np.zeros(0)

        for _ in range(1000):
            x, _ = next(train_iterator)

            real_imgs = x.cuda()
            # Train Discriminator
            z = torch.randn(batch_size, latent_dim, 1, 1).to(device)
            fake_imgs = generator(z)

            real_labels = torch.ones(batch_size).to(device)
            fake_labels = torch.zeros(batch_size).to(device)

            # Real images loss
            real_loss = criterion(discriminator(real_imgs), real_labels)
            # Fake images loss
            fake_loss = criterion(discriminator(fake_imgs.detach()), fake_labels)
            d_loss = real_loss + fake_loss

            optimiser_D.zero_grad()
            d_loss.backward()
            optimiser_D.step()

            # Train Generator
            g_loss = criterion(discriminator(fake_imgs), real_labels)

            optimiser_G.zero_grad()
            g_loss.backward()
            optimiser_G.step()

            steps += 1

            loss_arr_D = np.append(loss_arr_D, d_loss.item())
            loss_arr_G = np.append(loss_arr_G, g_loss.item())

            if steps >= 50000:
                break

        print(f"Steps: {steps}, Loss D: {loss_arr_D.mean():.3f}, Loss G: {loss_arr_G.mean():.3f}")

        # Sample model and visualize results
        generator.eval()
        z = torch.randn(64, latent_dim, 1, 1).cuda()
        samples = generator.sample(z).cpu().detach()
        plt.imshow(torchvision.utils.make_grid(samples).cpu().numpy().transpose(1, 2, 0), cmap=plt.cm.binary)
        plt.show()
        epochs += 1
        generator.train()
        if epochs % step_size == 0:
            scheduler_gen.step()
            scheduler_disc.step()
            
    
    # define directories
    real_images_dir = 'real_images'
    generated_images_dir = 'generated_images'
    num_samples = 10000 # do not change

    # create/clean the directories
    def setup_directory(directory):
        if os.path.exists(directory):
            !rm -r {directory} # remove any existing (old) data
        os.makedirs(directory)

    setup_directory(real_images_dir)
    setup_directory(generated_images_dir)

    # generate and save 10k model samples
    num_generated = 0
    while num_generated < num_samples:

        # sample from your model, you can modify this
        z = torch.randn(64, latent_dim, 1, 1).cuda()
        samples_batch = generator.sample(z).cpu().detach()

        for image in samples_batch:
            if num_generated >= num_samples:
                break
            save_image(image, os.path.join(generated_images_dir, f"gen_img_{num_generated}.png"))
            num_generated += 1

    # save 10k images from the CIFAR-100 test dataset
    num_saved_real = 0
    while num_saved_real < num_samples:
        real_samples_batch, _ = next(test_iterator)
        for image in real_samples_batch:
            if num_saved_real >= num_samples:
                break
            save_image(image, os.path.join(real_images_dir, f"real_img_{num_saved_real}.png"))
            num_saved_real += 1
    
    # compute FID
    score = fid.compute_fid(real_images_dir, generated_images_dir, mode="clean")
    print(f"FID score: {score}")
    file = open("FID-Scores","a")
    file.write(f"{score}, ({b1},{b2}), g_lr: {g_lr}, d_lr: {d_lr}, latent_dim: {latent_dim}, step_size: {step_size}, gamma: {gamma}\n")
    file.close()
    return score
# Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Best trial
print("Best trial:")
print(study.best_trial)

 Param count: 598148
 - Base DCGAN: FID 313
 - WGAN with feature matching and gradient penalty: FID 282
 - WGAN + fmgp + all train transformations: FID 324
 - WGAN + gfmp + reduced transformations + betas .9-.999: FID 272
 - above + d-lr=0.0002, g-lr=0.0001 (prev other way round): FID 217
 - without shear/rotations + above: FID 219
 
Param Count: 595776
 - DCGAN with g_lr=0.0001 and d_lr==0.0002: FID 422?
 - DCGAN with StepLR both lrs equal: FID 111
 - {'g_lr': 0.0006814031697767739, 'd_lr': 0.00012139497245059714, 'latent_dim': 64, 'step_size': 9, 'gamma': 0.6555892021167088, 'b1': 0.49814188804932835, 'b2': 0.9890717565690195}: FID 95.94
 - {'g_lr': 0.0006400197806639131, 'd_lr': 0.0004680754143284617, 'latent_dim': 224, 'step_size': 9, 'gamma': 0.703364065823086, 'b1': 0.5120438113537247, 'b2': 0.9928494626639106} FID: 95.13
 - {'g_lr': 0.0009286904414305756, 'd_lr': 0.00040507457935635836, 'latent_dim': 160, 'step_size': 9, 'gamma': 0.28468433714087593, 'b1': 0.4815824307625205, 'b2': 0.9587758651885724} - FID: 97.3
 - {'g_lr': 0.00036455004741175876, 'd_lr': 0.000739064082810268, 'latent_dim': 160, 'step_size': 10, 'gamma': 0.49525170974763577, 'b1': 0.4984104132166096, 'b2': 0.9996938161836509} FID: 97.25
 - {'g_lr': 0.0006305403183965492, 'd_lr': 0.00024562829361544024, 'latent_dim': 64, 'step_size': 6, 'gamma': 0.8982742062774087, 'b1': 0.512029309540369, 'b2': 0.9853474704544104} FID: 91.94

Param Count: 891968

- {'g_lr': 0.00037579454545191366, 'd_lr': 0.00013508523644514142, 'latent_dim': 64, 'step_size': 13, 'gamma': 0.32895807737962646} FID: 83.36
- {'g_lr': 0.0003813418582686221, 'd_lr': 0.00028307216565527947, 'latent_dim': 128, 'step_size': 10, 'gamma': 0.6912483540642389} FID: 91.43
- {'g_lr': 0.0008713135600597669, 'd_lr': 0.0002442567087266487, 'latent_dim': 192, 'step_size': 7, 'gamma': 0.4740671338389559} FID: 86.90
- {'g_lr': 0.00036916284893116796, 'd_lr': 0.000603707872600647, 'latent_dim': 64, 'step_size': 12, 'gamma': 0.7924788560314879}. FID: 91.06
- {'g_lr': 0.00036916284893116796, 'd_lr': 0.000603707872600647, 'latent_dim': 64, 'step_size': 12, 'gamma': 0.7924788560314879}. FID: 91.06
- {'g_lr': 0.0008505944870268719, 'd_lr': 0.00013707522718043292, 'latent_dim': 192, 'step_size': 12, 'gamma': 0.5506931925857736} FID: 83.92
- {'g_lr': 0.0006136210845775221, 'd_lr': 0.0002139380113632377, 'latent_dim': 96, 'step_size': 7, 'gamma': 0.7072523890037579} FID 86.44

discriminator lr <= g_lr
 
 - {'g_lr': 0.0007340907919889216, 'd_lr': 0.0001927856548861613, 'latent_dim': 128, 'step_size': 19, 'gamma': 0.5373091857650582}. Best is trial 0 with value: 85.10373456772481. FID 85.1
 - {'g_lr': 0.0005145033935415546, 'd_lr': 0.00010229425253813181, 'latent_dim': 128, 'step_size': 20, 'gamma': 0.8714934386353768} FID 87.4
 - {'g_lr': 0.0009417594819548194, 'd_lr': 0.00027712121409213965, 'latent_dim': 64, 'step_size': 7, 'gamma': 0.2705894174461227} FID: 86.28
 - {'g_lr': 0.0006511206888448697, 'd_lr': 0.0002183311745326889, 'latent_dim': 64, 'step_size': 9, 'gamma': 0.735978442880421} FID 87.18

- {'g_lr': 0.00043838182732142735, 'd_lr': 0.00016770962354068362, 'latent_dim': 64, 'step_size': 14, 'gamma': 0.7588884139473471} FID 88.98
- {'dropout': 0.24974735461990838}: 85.74
- {'dropout': 0.13642056472432817}: 86.73
- {'dropout': 0.30276459406217154}: 88.85
- {'dropout': 0.2518147586184857}. 89.61
- {'dropout': 0.13309362888176604} 84.66
 - 0.22450924820171564: 85.55

**967,072 params**
- transforms + tuned learning rates and latent_dim: FID 98.29
- transforms + 0.0002 lr + 100 latent_dim: FID 84.88
- Resize only + above: FID 58.77!! -> 56.48
- {'g_lr': 0.00047011897412151387, 'd_lr': 0.00032888835823867305, 'step_size': 14, 'gamma': 0.13605607382582346} FID 52.98736270293523
- {'g_lr': 0.0004646569322993303, 'd_lr': 0.0006323606861306212, 'step_size': 5, 'gamma': 0.8797304441483801} FID 51.49
- {'g_lr': 0.0003742610055813428, 'd_lr': 0.0004009095378355368, 'step_size': 5, 'gamma': 0.8910603576191873} FID 51.59 low step size + high gamma or vice versa works quite well
- {'g_lr': 0.00024730537498181265, 'd_lr': 0.0004064663394286604, 'step_size': 12, 'gamma': 0.14786319216080435} FID 54.92
- {'g_lr': 0.00010054020620311011, 'd_lr': 0.00010364349860835665, 'step_size': 9, 'gamma': 0.19138821661905886} FID 66.02, low base LRs + low gamma and step size all combine badly
- {'g_lr': 0.0005904097365207748, 'd_lr': 0.0009556906871456465, 'step_size': 19, 'gamma': 0.3100246769446714} fid 50.47
- {'g_lr': 0.0005502189910110365, 'd_lr': 0.00012141854619756989, 'step_size': 5, 'gamma': 0.29843701854660687} FID 49.16
- {'g_lr': 0.0005132927362642562, 'd_lr': 0.00012275457879686526, 'step_size': 10, 'gamma': 0.5822640412702591} FID 48.65
- {'g_lr': 0.00039890686782809696, 'd_lr': 0.0007170630663625129, 'step_size': 14, 'gamma': 0.10723648399528232} FID 51.29
- {'g_lr': 0.0005749455818234153, 'd_lr': 0.0002958197122217904, 'step_size': 9, 'gamma': 0.40123837184511213} fid 48.63
- {'g_lr': 0.0004465779109723407, 'd_lr': 0.0008005048604263282, 'step_size': 10, 'gamma': 0.15622517275303516} FID 49.91
- {'g_lr': 0.0004301786361677816, 'd_lr': 0.0004733102504837982, 'step_size': 11, 'gamma': 0.8248953417036737} FID 49.85
 - {'g_lr': 0.00029525225977134736, 'd_lr': 0.000703981679873739, 'step_size': 10, 'gamma': 0.6751003500761057} low fid, likely due to huge gap betwen d_lr and g_lr
  - {'g_lr': 0.0006006667193773175, 'd_lr': 0.0005898293581241078, 'step_size': 6, 'gamma': 0.2797635525687869 FID 45.62

- {‘g_lr': 0.0006138031334792676, 'd_lr': 0.0005625836234903569, 'step_size': 5, 'gamma': 0.3300497180274462} FID47.77

- {‘g_lr': 0.0006573019293906829, 'd_lr': 0.0006111283690657075, 'step_size': 5, 'gamma': 0.3104907558908947} FID48.45
 - {'g_lr': 0.0007508166505062501, 'd_lr': 0.000453966691674159, 'step_size': 6, 'gamma': 0.20783980641627642} FID 44.46
 - {'g_lr': 0.0005925841689063689, 'd_lr': 0.0003828441036673148, 'step_size': 7, 'gamma': 0.4680577304310266} FID 46.29
  - {'g_lr': 0.0006828838288502215, 'd_lr': 0.000275880375280674, 'step_size': 7, 'gamma': 0.1152882158531075} FID 44.55
 - {'g_lr': 0.0007707339926668834, 'd_lr': 0.00027338506365327066, 'step_size': 15, 'gamma': 0.1037305611657213} fid 47.38